# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

In [1]:
'''
Your Input
'''
'''
多線程爬蟲是藉由不同tread來同步執行爬蟲
非同步爬蟲是藉由爬蟲排程將空白的地方(request花時間...)拿去做別的事情以減低整體時間，只會使用1個thread
'''


'\nYour Input\n'

In [35]:
import requests
import _thread
import time
import re

import requests
from bs4 import BeautifulSoup


T1=time.time()
urls = ['https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/job/p/236292934']

def normal_scrapy(url) :
    r = requests.get(url)
    r.encoding = 'utf-8'

    soup = BeautifulSoup(r.text, "lxml")
    reply = soup.find_all('section')[1].find('div',attrs={'style':re.compile('padding-top')})

    comment=[]
    #print(reply.find('div'))
    for d in reply.find_all('div',attrs={'data-index':re.compile('[0-9]')}):
        try :
            floor = d.find('a' , attrs={'class':'sc-7fxob4-6 neiWc'}).text.replace(' ','')
        except :
            continue
        com=[]
        for c in d.find('div',attrs={'class':'phqjxq-0 fQNVmg'}).find_all('span') :
            if c.string != None :
                com.append(c.string)
        if com !=[]:
            comment.append(floor+':'+com[0])
    
    #for coment in comment : print(coment,'\n\n')
    print('scrapy url : ', url  )
#print(soup)
for url in urls :
    normal_scrapy(url)

print("花費:" + str(time.time() - T1) + "秒")

scrapy url :  https://www.dcard.tw/f/stock/p/236285134
scrapy url :  https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/mood/p/236289487
scrapy url :  https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/stock/p/236285134
scrapy url :  https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/mood/p/236289487
scrapy url :  https://www.dcard.tw/f/job/p/236292934
花費:1.2720270156860352秒


In [36]:

T1=time.time()
urls = ['https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/job/p/236292934']

def dcard_scrapy (thread_name , url):
    r = requests.get(url)
    r.encoding = 'utf-8'

    soup = BeautifulSoup(r.text, "lxml")
    reply = soup.find_all('section')[1].find('div',attrs={'style':re.compile('padding-top')})

    #print(reply.find('div'))
    for d in reply.find_all('div',attrs={'data-index':re.compile('[0-9]')}):
        comment=[]
        try :
            floor = d.find('a' , attrs={'class':'sc-7fxob4-6 neiWc'}).text.replace(' ','')
        except :
            continue
        for c in d.find('div',attrs={'class':'phqjxq-0 fQNVmg'}).find_all('span') :
            if c.string != None :
                comment.append(c.string)
        #print(thread_name,'-->',floor,':\n',comment[0].replace(' ',''),"\n")
    print('scrapy url : ' ,url)
    if(url == 'https://www.dcard.tw/f/job/p/236292934'):
        print("花費:" + str(time.time() - T1) + "秒")
        

i=1
for url in urls:
    _thread.start_new_thread(dcard_scrapy,('thread '+str(i),url))
    i=i+1

time.sleep(2)
#print("花費:" + str(T2 - T1) + "秒")

scrapy url :  scrapy url : https://www.dcard.tw/f/stock/p/236285134
 https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/stock/p/236285134
scrapy url :  https://www.dcard.tw/f/mood/p/236289487
scrapy url :  https://www.dcard.tw/f/stock/p/236282709
scrapy url :  https://www.dcard.tw/f/mood/p/236289487
scrapy url :  https://www.dcard.tw/f/job/p/236292934
花費:1.0234124660491943秒


In [37]:
from aiohttp import ClientSession
import asyncio
import nest_asyncio
nest_asyncio.apply()

T1=time.time()
urls = ['https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/stock/p/236285134','https://www.dcard.tw/f/stock/p/236282709',
       'https://www.dcard.tw/f/mood/p/236289487','https://www.dcard.tw/f/job/p/236292934']

async def fetch(link,session):
    async with session.get(link) as response:
        r = await response.text()
    
    soup = BeautifulSoup(r, "lxml")
    reply = soup.find_all('section')[1].find('div',attrs={'style':re.compile('padding-top')})

    #print(reply.find('div'))
    for d in reply.find_all('div',attrs={'data-index':re.compile('[0-9]')}):
        comment=[]
        try :
            floor = d.find('a' , attrs={'class':'sc-7fxob4-6 neiWc'}).text.replace(' ','')
        except :
            continue
        for c in d.find('div',attrs={'class':'phqjxq-0 fQNVmg'}).find_all('span') :
            if c.string != None :
                comment.append(c.string)
        #print(thread_name,'-->',floor,':\n',comment[0].replace(' ',''),"\n")
    print('scrapy url : ' ,url)
    
async def main(loop):
    async with ClientSession() as session:
        tasks = [loop.create_task(fetch(url, session)) for url in urls]
        await asyncio.gather(*tasks)
        

loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
#loop.close()
print("花費:" + str(time.time() - T1) + "秒")

scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
scrapy url :  https://www.dcard.tw/f/job/p/236292934
花費:0.849513053894043秒
